### Script to use when an agency changes their route_ids
* Requires the old routes.txt & trips.txt, as well as the new routes.txt & trips.txt
* It replaces existing routes with the previous route_ids and keeps the existing route_id if a route is new
* The results are a routes_NEW.txt and trips_NEW.txt 

In [1]:
import pandas as pd
import numpy as np

In [54]:
# old update (I want these route_ids)
filepath = r'bbb'
# new update (These route_ids will be replaced)
filepathBad = r'bbb-new'

# load old file
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    return pd.read_csv(f)

# load new (bad) file
def loadBad(fName): 
    f = filepathBad + "/" + "{}.txt".format(fName)
    return pd.read_csv(f)

# save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepathBad + '/' + "{}_FIXED.txt".format(fName), sep =',', index=False)
    return print("Saved {}_FIXED.txt to {}".format(fName,filepathBad))

# load good (old) GTFS files
routesOld = load('routes')
tripsOld = load('trips')

# load new (bad) GTFS files
routesNew = loadBad('routes')
tripsNew = loadBad('trips')

In [55]:
# merge the two routes dataframes together based on route_short_name
newRouteMerge = pd.merge(routesNew,routesOld,on=['route_short_name'],how='left')
# add the ids for the new routes to the field with the old routes
newRouteMerge.loc[newMerge['route_id_y'].isna(), 'route_id_y'] = newMerge['route_id_x']
# write the new routes.txt file
keepRoutes = pd.DataFrame({
    'route_id' : newMerge.route_id_y.astype('int32'),
    'agency_id' : routesNew.agency_id,
    'route_short_name' : routesNew.route_short_name,
    'route_long_name': routesNew.route_long_name,
    'route_desc': routesNew.route_desc,
    'route_type': routesNew.route_type,
    'route_url': routesNew.route_url,
    'route_color': routesNew.route_color,
    'route_text_color': routesNew.route_text_color 
})
# save new .txt file
save(keepRoutes,'routes')




Saved routes_FIXED.txt to bbb-new


In [56]:
# merge the neweRouteMerge df with tripNew based on route_id from new data
newTripMerge = pd.merge(tripsNew,newMerge,left_on=['route_id'],right_on=['route_id_x'],how='outer')
# write new trips.txt file
keepTrips = pd.DataFrame({    
    'route_id': newTripMerge.route_id_y.astype('int32'), 
    'service_id': tripsNew.service_id,
    'trip_id': tripsNew.trip_id,
    'trip_headsign': tripsNew.trip_headsign,
    'trip_short_name': tripsNew.trip_short_name,
    'direction_id': tripsNew.direction_id,
    'block_id': tripsNew.block_id,
    'shape_id': tripsNew.shape_id,
    'wheelchair_accessible': tripsNew.wheelchair_accessible,
    'bikes_allowed': tripsNew.bikes_allowed    
})
# save new .txt file
save(keepTrips,'trips')


Saved trips_FIXED.txt to bbb-new
